In [0]:
spark.conf.set(
  "fs.azure.account.key.goodreadsreviews60300832.dfs.core.windows.net",
  "dj khaledddd anotha one"
)

In [0]:
storage_acct = "goodreadsreviews60300832"  
container    = "lakehouse"

gold_path        = f"abfss://{container}@{storage_acct}.dfs.core.windows.net/gold"
curated_path     = f"{gold_path}/curated_reviews"
features_v1_path = f"{gold_path}/features_v1"


In [0]:
# Consistent time handling for date parsing/formatting
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
spark.conf.set("spark.sql.session.timeZone", "UTC")


In [0]:
from pyspark.sql import functions as F

def read_delta_or_parquet(path: str):
    try:
        return spark.read.format("delta").load(path)
    except Exception:
        return spark.read.format("parquet").load(path)

# 1) LOAD curated (no silver)
df = read_delta_or_parquet(curated_path)
print(f"Loaded: {curated_path}")
df.printSchema()
display(df.limit(10))

Loaded: abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/gold/curated_reviews
root
 |-- review_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_text: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- date_added: string (nullable = true)



review_id,book_id,title,author_id,name,user_id,rating,review_text,language_code,n_votes,date_added
8e9c4a053fee8a02a5bfc29455ee3d3c,13539044,The Silver Linings Playbook,1251730,Matthew Quick,4cb85124ba4822edd1547834854b9523,5,Read it after I saw movie still loved it!!,eng,0,Fri Feb 07 20:41:44 -0800 2014
d74e0371d79ac8a59f282e7d8aa55717,22726728,"Adjournment (Fate, #1)",8261567,Emersyn Vallis,46aeb1e5b0c79caaed4082e2fdcdc34c,4,"4.5 stars for Emersyn Vallis's debut novel. I LOVED it!!!!! I can not remember the last time a book had me laughing so hard! I had a hard time getting into the story because the beginning was a little slow for me but once the ball started rolling I couldn't put the book down. The story centers around Sydney Chandler and her first love Simon McAllister. At graduation Simon committed the ultimate betrayal and left Sydney brokenhearted. It was then that she decided she was going to change the path of her life and move to New York. The two ran into each other a few times over the last few years but Sydney always acted like she hated Simon. But when Sydney's parents renew their wedding vows and Sydney and Simon are forced to spend some time together will she be able to keep that wall up? Adjournment has a cast a characters that will keep you laughing your butt off!!! You just can't help but fall in love with all of them. The witty banter is hilarious and the various scenarios that the characters find themselves caught up in will having you rolling! You can't turn the pages fast enough to see what these guys will come up with next. It is pranks galore!! The chemistry between Simon and Sydney just jumps off the page at you. These two are fire when they get started. I really enjoyed this book and I think it is amazing for a debut novel. I can not wait to see what the author has in store for book 2, which is expected to release in September! Bravo Ms. Vallis",eng,1,Sun Jun 22 06:36:06 -0700 2014
a307fc478ca1649e7041282edc0006d2,23599844,"One Night with a Santini (The Santinis, #8)",223787,Melissa Schroeder,12f978ace06b4b1b9dcd579c4a93b0db,5,Another great addition to The Santinis series.,eng,0,Fri Mar 06 18:47:40 -0800 2015
574205e94619994e119a098d166141de,22549630,Duisternis,2383,Gillian Flynn,e30428def7cf29ab20eebc67a90d7dca,5,"8.5 Het tweede boek wat ik van Gillian Flynn heb gelezen en wauw weer heel goed! Af en toe moest ik mijzelf uit het boek trekken, want dan zat ik te diep in het verhaal. Zeker een aanrader, een nieuwe favoriet!",nl,0,Fri Jul 11 03:41:27 -0700 2014
bc0098d14d048650532222954f905ea9,9464733,Beauty Queens,2526,Libba Bray,e96efee59e458d3da595523e09d4de36,3,"Actual Rating - 3.5 I wasn't sure what I was going to rate this. Mainly because some parts of Beauty Queen is utterly ridiculous, but some parts also made me laugh out loud. This book would also satisfy any feminists needs, as it questioned many roles within society. But, the girls still loved their pretty things, and their clothes and make up, which is good, but they became more aware of feminist ideas. There was some really lovely character development, which was unexpected. These girls was using their skills to survive on an island and just proves they are not just the stereotypical airheads. The friendship between the girls in Beauty Queens was also really nice to read, and it was nice that they all helped each other realise that they don't always need to be accepted by society, but be accepted by themselves. Aspects of this book left me thinking 'are you serious?. But Libba Bray addressed many serious issues in a funny and light hearted way. Overall, it was an easy, humorous read which I enjoyed despite many crazy and ridiculous aspects.",eng,1,Sun Jun 16 10:58:56 -0700 2013
30f47626f8f99d03cd456019f08cbe59,140082,"Club Dead (Sookie Stackhouse, #3)",17061,Charlaine Harris,0842a3ab06f46aaf3521b4f9bf6995aa,3,"I felt like this book is one of those ""in between"" books. It is interesting and fun to read, but you can just tell that it is l

In [0]:
# 2) CLEANING
def has(c): return c in df.columns

# 2.1 Required non-null
required = [c for c in ["rating","review_text","book_id","author_id"] if has(c)]
if required:
    df = df.dropna(subset=required)

# 2.2 Duplicates: by review_id else (user_id, book_id)
if has("review_id"):
    df = df.dropDuplicates(["review_id"])
elif has("user_id") and has("book_id"):
    df = df.dropDuplicates(["user_id","book_id"])

# 2.3 Normalize/trim text fields
if has("review_text"):
    df = (df
          .withColumn("review_text", F.regexp_replace("review_text", r"[\u0000-\u001F\u007F]", " "))
          .withColumn("review_text", F.trim(F.col("review_text")))
          .withColumn("review_text", F.lower(F.col("review_text"))))

for c in ["book_id","author_id","user_id","review_id","title","name","language_code"]:
    if has(c):
        df = df.withColumn(c, F.trim(F.col(c)))

# 2.4 Drop very short reviews (<10 words)
if has("review_text"):
    df = df.withColumn("_wc", F.size(F.split(F.col("review_text"), r"\s+"))) \
           .filter(F.col("_wc") >= 10) \
           .drop("_wc")

# 2.5 Data types
if has("rating"):
    df = df.withColumn("rating", F.col("rating").cast("double"))

if has("n_votes"):
    df = df.withColumn("n_votes", F.col("n_votes").cast("long"))

for c in ["book_id","author_id","user_id","review_id","title","name","language_code"]:
    if has(c):
        df = df.withColumn(c, F.col(c).cast("string"))

# 2.6 Dates → keep ONLY DATE as `date_added`
# Parse common patterns from original string `date_added`
if has("date_added") and dict(df.dtypes).get("date_added") == "string":
    parsed_ts = F.coalesce(
        F.to_timestamp("date_added"),
        F.to_timestamp("date_added", "yyyy-MM-dd HH:mm:ss"),
        F.to_timestamp("date_added", "yyyy-MM-dd"),
        F.to_timestamp("date_added", "MM/dd/yyyy HH:mm:ss"),
        F.to_timestamp("date_added", "MM/dd/yyyy"),
        F.to_timestamp("date_added", "EEE MMM dd HH:mm:ss Z yyyy")  # Tue Mar 17 13:18:31 -0700 2015
    )
    df = df.withColumn("date_added", F.to_date(parsed_ts))  # overwrite as DATE

# 2.7 Validate numeric ranges
if has("rating"):
    df = df.filter((F.col("rating") >= 1.0) & (F.col("rating") <= 5.0))
if has("n_votes"):
    df = df.filter(F.col("n_votes") >= 0)

# 2.8 Drop unused columns if present
drop_candidates = [c for c in ["html_body","temp_flag","_rescued_data","raw_json","ingest_ts"] if has(c)]
if drop_candidates:
    df = df.drop(*drop_candidates)

# Optional rename `name` → `author_name`
if has("name"):
    df = df.withColumnRenamed("name", "author_name")

# COMMAND ----------

# 3) FEATURES
# 3.1 review_length_words
if has("review_text"):
    df = df.withColumn("review_length_words",
                       F.size(F.split(F.col("review_text"), r"\s+")).cast("int"))

# 3.2 review_length_chars
if has("review_text"):
    df = df.withColumn("review_length_chars", F.length(F.col("review_text")).cast("int"))

# 3.3 Aggregate by book_id
if has("book_id"):
    agg = df.groupBy("book_id").agg(
        F.count(F.lit(1)).alias("book_review_count"),
        *( [F.avg("rating").alias("book_avg_rating")] if has("rating") else [] )
    )
    df = df.join(agg, on="book_id", how="left")


In [0]:
# 4) SELECT FINAL COLUMNS (DATE ONLY, no extra timestamp/ISO cols)
final_cols = [c for c in [
    "review_id","user_id","book_id","author_id","author_name",
    "title","language_code","n_votes",
    "rating","review_text",
    "review_length_words","review_length_chars",
    "date_added",                 # DATE only
    "book_review_count","book_avg_rating"
] if has(c)]

final_df = df.select(*final_cols) if final_cols else df

print("Final schema:")
final_df.printSchema()
display(final_df.limit(10))


Final schema:
root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- author_name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- review_text: string (nullable = true)
 |-- review_length_words: integer (nullable = false)
 |-- review_length_chars: integer (nullable = true)
 |-- date_added: date (nullable = true)
 |-- book_review_count: long (nullable = true)
 |-- book_avg_rating: double (nullable = true)



review_id,user_id,book_id,author_id,author_name,title,language_code,n_votes,rating,review_text,review_length_words,review_length_chars,date_added,book_review_count,book_avg_rating
04af78ec9ae6dfa8f685450395f736f6,a8c9175daa5a7dd95750ae0261b4838d,15507958,281810,Jojo Moyes,"Me Before You (Me Before You, #1)",eng,0,5.0,probably the best book of its kind out there. i loved every minute of this book.,16,80,2016-06-02,7944,4.330941591137965
b2ecb10efea6b86105105318ef408a8e,826f7d7a842759549130072896d999c2,17331828,2995873,Julie Kagawa,"Talon (Talon, #1)",eng,0,4.0,this book was totally amazing!! first things first the cover is so simple but it looks really pretty like that!! oh and it's by julie kagawa!!! why would anyone not want to read one of her books?? her iron fey series was amazing and this was good as well but i kinda think it paled in comparison to her iron fey series!!,62,320,2014-08-26,618,3.5906148867313914
2dc9f631b875f357a46229b87e56de83,c007d5eb99b3804b3f4a9a764eee83e7,13519844,4951964,Shelly Crane,"Independence (Significance, #4)",eng,0,4.0,"perfect ending to the significance series. i'm actually looking forward to see the movie, although it's still in pre-production. i absolutely loved all the characters minus beck,..she annoyed me up until the end. i would definitely read this series again.",40,255,2013-11-24,200,4.29
4c600d19e6be5097125d6e628a0f27d1,8c6b2fa6608814033375191a10f4eb98,20309175,4208569,Rainbow Rowell,My True Love Gave to Me: Twelve Holiday Stories,eng,12,4.0,"this was a lovely short story collection that introduced me to a number of authors that i haven't read before. midnights by rainbow rowell- 5 stars- this was the first thing i've ever read by rainbow, and it left me wanting more from her! i loved this story. it personally tugged on my heartstrings due to some nostalgia surrounding a high school crush, and the fact that this short of a story could tap into that is a testament to how lovely it was. the lady and the fox by kelly link- 3 stars- i enjoyed elements of this story, but overall didn't enjoy the ending. i really liked the first half though. angels in the snow by matt de la pena- 3 stars- elements of this story rubbed me the wrong way. i don't enjoy a lot of swearing in my stories (even though i curse like a sailor in real life at times :) ) and this just had too much for me, which made it hard for me to connect to the main character. polaris is where you'll find me by jenny han- 3 stars- this left me wanting more in a bad way. i needed 5-10 more pages of story and i would have really enjoyed this! it's a yuletide miracle, charlie brown by stephanie perkins- 5 stars- no surprise here. i loved this. i connected to the characters, i found it to be believable, and the romance was cute!!! your temporary santa by david levithan- 3 stars- i love that this had lgbt characters in it, as is david levithan's style, but i didn't really connect to the story, sadly. krampuslauf by holly black- 2 stars- this was my first holly black and i was not impressed. i did not like the direction it took and i didn't feel it flowed with the rest of the stories. what the hell have you done, sophie roth by gayle forman- 4 stars- this was also my first gayle forman and i was impressed. another believable story and romance. beer buckets and baby jesus by myra mcentire- 4 stars- this was cute and felt very much like a hallmark movie or a christmas themed episode of a show like gilmore girls, in a good way! welcome to christmas, ca by kiersten white- 5 stars- i found this story the perfect mix of magical realism with contemporary, with a nice dash of family dynamics. so cute! star of bethlehem by ally carter- 4 stars- this story didn't finish as strongly as i felt it could have, but it was very enjoyable. again, another example of a hallmark movie! a little unbelievable, but still cute. the girl who woke the dreamer by laini taylor- 5 stars- this was my first laini taylor, and i have heard so much about her writing befo

In [0]:
# 5) SAVE → gold/features_v1 (Delta)
(final_df
 .write
 .format("delta")
 .mode("overwrite")
 .option("overwriteSchema","true")
 .save(features_v1_path))

print(f"Saved to: {features_v1_path}")

Saved to: abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/gold/features_v1


In [0]:
# 6) RELOAD & VERIFY
ver = spark.read.format("delta").load(features_v1_path)
print("Reloaded from:", features_v1_path)
ver.printSchema()

print("Record count →", ver.count())
display(ver.limit(20))

if "rating" in ver.columns:
    display(ver.select(
        F.count(F.lit(1)).alias("n"),
        F.min("rating").alias("min_rating"),
        F.max("rating").alias("max_rating"),
        F.avg("rating").alias("avg_rating")
    ))

cols = [c for c in ["book_id","book_review_count","book_avg_rating"] if c in ver.columns]
if cols:
    display(ver.select(*cols).orderBy(F.desc_nulls_last("book_review_count")).limit(20))

Reloaded from: abfss://lakehouse@goodreadsreviews60300832.dfs.core.windows.net/gold/features_v1
root
 |-- review_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- author_id: string (nullable = true)
 |-- author_name: string (nullable = true)
 |-- title: string (nullable = true)
 |-- language_code: string (nullable = true)
 |-- n_votes: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- review_text: string (nullable = true)
 |-- review_length_words: integer (nullable = true)
 |-- review_length_chars: integer (nullable = true)
 |-- date_added: date (nullable = true)
 |-- book_review_count: long (nullable = true)
 |-- book_avg_rating: double (nullable = true)

Record count → 13556235


review_id,user_id,book_id,author_id,author_name,title,language_code,n_votes,rating,review_text,review_length_words,review_length_chars,date_added,book_review_count,book_avg_rating
bb3a7bea0e928be8025505c84031eafd,8273fc29780c84559cbd533fc969c0a6,138068,16461,David Wiesner,Free Fall,,0,4.0,it's in the spectrum between in the night kitchen and the imagine a... books by sarah l. thomson and rob gonsalves. http://www.pussreboots.pair.com/blog/...,22,158,2015-06-22,30,3.9
6001679b44cca7c6f1f64d49f5eaa2c8,0d11de2091a5c80f6f0b17b634716217,18877100,7306558,James Flerlage,Before Bethlehem,eng,3,5.0,"an awesome historical novel detailing the details surrounding the birth of jesus told from the viewpoint of joseph's son, james!!! a very believable work of fiction!!!",26,167,2014-01-03,10,3.3
d1a98dded13e6b6dfe7c5feb2899ed22,deb48a9c08a902329fd437f3c5a770cd,13539044,1251730,Matthew Quick,The Silver Linings Playbook,eng,0,4.0,"i'm glad that i saw this movie first because i really liked the movie. i'm fairly certain i would not have liked it if i'd read this first (i have similar feelings about bridget jones's diary). i was unsure about how i felt about it until i got closer to the end. i think that this book, like life of pi (two other book references in one review...i'm on a roll), crept up on me and i went from sort of liking it to being really glad that i read it.",91,448,2013-07-13,1612,3.9727047146401984
7109bd0cc5131e3d9c1b69f4c82109ba,cdda5fbf8bf4344341e50216a9568726,26777770,13625621,Sloan Storm,"Surge (Fastlane Series, #1)",eng,0,4.0,"*** link for the review*** http://ceresbooksworld.com/2016/01/22... first of all i was born in le mans in france, i don't know if this city says something to you but there is the 24 hours of le mans. i love formula one and cars so this book was for me, obviously. this is the story of ava, a young nurse and dyson, a formula one pilot with a big mouth. ava is a sweet girl who had a hard life. she's raising her nephew, simon, alone and trying to give him a better life. she must left her dreams of career aside to provide simon with better study. during her day at the clinic, she's meeting dyson vix, a formula one's pilot and bad boy, hot and sexy who loves women mostly in his bed. she's attracted by him but she's always thinking about her nephew and don't want a man in their life. simon is her top priority. she will end up taking a job with the team and working with dyson. it will not be simple and a person from her past will come back sooner than later. ava is a really good character, she's selfless, very kind, and it's amazing how problems always find her. i like seeing her interact with dyson, they are so funny, sweet and hot. even if the sport is formula one and fast, the book is a slow pace romance. dyson and ava attraction are growing slowly. dyson is enjoyable, yes he has some problems and he doesn't open his heart but i like it when he takes care of ava. he's a sweet boy with anger and authority problems. simon is a sweet boy, i love his relation with ava and he didn't have chance with other kids. jillian is also a character i like, she's ava's best friend and she's helping her with simon. jillian has always good advice for ava and their relation is important in the story. this is the first sloan storm's book i read and it was awesome, i like her writing style, it's simple, hot and fast. i would have liked that the passages on dyson are a little more thorough, as much on his feelings to ava as on his ""disease"". surge is the first book in the series and i'm impatient to read the next one. it makes me think of fast track series by erin mccarthy.",402,2099,2015-09-27,34,4.147058823529412
6aaad3c5a7ad57aea688e07186279cd7,5f3e672cf462988850164a4049d7bea1,13125947,1304470,Ally Condie,"Reached (Matched, #3)",,0,4.0,a great ending to a great trilogy but xander and lei love story feel kind of rushed to me.,19,90,2012-09-04,1643,3.310407790626902
da6b1380282a564f09100b5fe496c101,592111d89282f9767e6220fe9

n,min_rating,max_rating,avg_rating
13556235,1.0,5.0,3.899646398871073


book_id,book_review_count,book_avg_rating
11870085,17330,4.392498557414887
11870085,17330,4.392498557414887
11870085,17330,4.392498557414887
11870085,17330,4.392498557414887
11870085,17330,4.392498557414887
11870085,17330,4.392498557414887
11870085,17330,4.392498557414887
11870085,17330,4.392498557414887
11870085,17330,4.392498557414887
11870085,17330,4.392498557414887
